# Lab 4: End-to-End Supply Chain Security

In this capstone lab, you'll build a complete secure software supply chain for a sample application.
You'll implement controls at each stage of the pipeline and validate the entire system against attacks.

The sample application will be a simple Python package, but the security concepts apply broadly.

## Setup

First, let's set up our end-to-end pipeline components:

In [1]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Set
from datetime import datetime
import hashlib
import hmac
import json
import os
from pathlib import Path

# Import key classes we defined in previous labs
%run 1_source_control_system_attack.ipynb
%run 2_build_system_compromises.ipynb
%run 3_software_distribution_security.ipynb

@dataclass 
class SourceStage:
    """Represents source control stage of pipeline"""
    repo: GitRepo
    trusted_keys: Set[str]
    signed_commits: List[SignedGitCommit]
    
@dataclass
class BuildStage:
    """Represents build stage of pipeline"""
    system: BuildSystem
    environments: Dict[str, BuildEnvironment]
    artifacts: Dict[str, BuildArtifact]
    
@dataclass
class DistributionStage:
    """Represents distribution stage of pipeline"""
    repository: Repository
    transparency_log: TransparencyLog
    
@dataclass 
class SecureSupplyChain:
    """Complete end-to-end secure supply chain"""
    source: SourceStage
    build: BuildStage
    distribution: DistributionStage
    security_policies: Dict[str, dict]

# Set up test supply chain
supply_chain = SecureSupplyChain(
    source=SourceStage(
        repo=GitRepo(),
        trusted_keys=set(),
        signed_commits=[]
    ),
    build=BuildStage(
        system=BuildSystem(),
        environments={},
        artifacts={}
    ),
    distribution=DistributionStage(
        repository=Repository(
            name="secure-pypi",
            packages={},
            trusted_publishers=set()
        ),
        transparency_log=TransparencyLog(entries=[])
    ),
    security_policies={}
)

AttributeError: 'NoneType' object has no attribute 'hash'

AttributeError: 'NoneType' object has no attribute 'hash'

### Exercise 1: Security Policy Configuration

First, you'll implement the security policies that will govern the supply chain.
These policies should cover:
1. Commit signing requirements
2. Build environment controls
3. Package signing requirements
4. Distribution controls

The policies will be enforced by later functions. Focus on defining comprehensive rules.

In [ ]:
def configure_security_policies(chain: SecureSupplyChain) -> Dict[str, dict]:
    """
    Configure security policies for the supply chain
    
    Args:
        chain: The supply chain to configure
        
    Returns:
        Dictionary of security policies
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Test code
policies = configure_security_policies(supply_chain)

# Should have policies for each stage
assert "source" in policies
assert "build" in policies
assert "distribution" in policies

# Source policies
source = policies["source"]
assert "required_signed_commits" in source
assert "allowed_signing_keys" in source
assert source["required_signed_commits"] is True

# Build policies
build = policies["build"]
assert "hermetic_builds" in build
assert "allowed_base_images" in build
assert "artifact_signing" in build

# Distribution policies 
dist = policies["distribution"]
assert "package_signing" in dist
assert "transparency_log" in dist
assert dist["package_signing"]["required"] is True

print("Security policies configured successfully!")

<details>
<summary>Solution</summary>

```python
def configure_security_policies(chain: SecureSupplyChain) -> Dict[str, dict]:
    return {
        "source": {
            "required_signed_commits": True,
            "allowed_signing_keys": ["prod-sign-key-2024"],
            "protected_branches": ["main", "release-*"],
            "required_reviews": 2,
            "block_force_push": True
        },
        "build": {
            "hermetic_builds": True,
            "allowed_base_images": [
                "python:3.9-slim@sha256:abc123",
                "python:3.10-slim@sha256:def456"
            ],
            "artifact_signing": {
                "required": True,
                "algorithm": "sha256",
                "key_rotation_days": 90
            },
            "network_access": "blocked",
            "cache_validation": "sha256"
        },
        "distribution": {
            "package_signing": {
                "required": True,
                "algorithm": "sha256",
                "key_rotation_days": 90
            },
            "transparency_log": {
                "required": True,
                "min_attestations": 2
            },
            "allowed_registries": [
                "secure-pypi.example.com"
            ],
            "version_policy": {
                "immutable": True,
                "semver_required": True
            }
        }
    }
```
</details>


### Exercise 2: Secure Build Pipeline

Next, implement a function that processes artifacts through the secure pipeline:
1. Verify signed source commits
2. Build in hermetic environment
3. Generate SLSA provenance
4. Sign artifacts
5. Push to secure repository

This should use the security policies you defined above.

In [ ]:
def build_secure_artifact(
    chain: SecureSupplyChain,
    source_commit: SignedGitCommit,
    build_config: dict
) -> BuildArtifact:
    """
    Builds an artifact using the secure pipeline
    
    Args:
        chain: Configured supply chain
        source_commit: Signed commit to build from
        build_config: Build configuration
        
    Returns:
        Built and signed artifact
    """
    # YOUR CODE HERE
    pass


In [ ]:
# Test code
# Create test commit
commit = SignedGitCommit(
    hash="abc123",
    message="Test commit",
    timestamp=datetime.now(),
    author="test-dev",
    signature=b"valid-signature",
    author_public_key="test-key"
)

# Add to allowed keys
supply_chain.source.trusted_keys.add("test-key")

# Create build config
config = {
    "build_type": "python",
    "python_version": "3.9",
    "dependencies": ["pytest"]
}

# Build artifact
artifact = build_secure_artifact(supply_chain, commit, config)

# Verify artifact was built securely
assert artifact.metadata["source_commit"] == commit.hash
assert artifact.metadata["build_type"] == "hermetic"
assert "signature" in artifact.metadata
assert artifact.metadata["slsa_provenance"]

print("Secure build pipeline implemented successfully!")

<details>
<summary>Solution</summary>

```python
def build_secure_artifact(
    chain: SecureSupplyChain,
    source_commit: SignedGitCommit,
    build_config: dict
) -> BuildArtifact:
    # Verify commit signature
    if not verify_commit_signature(
        commit=source_commit,
        trusted_keys=chain.source.trusted_keys
    ):
        raise ValueError("Invalid commit signature")
        
    # Create hermetic build environment
    build_env = create_hermetic_environment(
        python_version=build_config["python_version"],
        dependencies=build_config["dependencies"]
    )
    
    # Verify environment meets policy
    if not verify_build_environment(
        env=build_env,
        policies=chain.security_policies["build"]
    ):
        raise ValueError("Build environment violates policy")
        
    # Perform build
    artifact = chain.build.system.build(
        source_commit=source_commit,
        build_env=build_env,
        config=build_config
    )
    
    # Generate SLSA provenance
    provenance = generate_slsa_provenance(
        artifact=artifact,
        source_commit=source_commit,
        build_env=build_env,
        build_config=build_config
    )
    
    # Sign artifact
    signature = sign_artifact(
        artifact=artifact,
        key=get_signing_key(chain)
    )
    
    # Add metadata
    artifact.metadata.update({
        "source_commit": source_commit.hash,
        "build_type": "hermetic",
        "signature": signature.hex(),
        "slsa_provenance": provenance.to_dict()
    })
    
    return artifact
```
</details>


### Exercise 3: Secure Distribution Pipeline

Now implement the distribution pipeline that publishes artifacts securely:
1. Verify artifact signatures & provenance
2. Update transparency log
3. Generate attestations
4. Publish to repository
5. Verify mirror consistency

This completes the end-to-end secure supply chain.

In [ ]:
def publish_secure_artifact(
    chain: SecureSupplyChain,
    artifact: BuildArtifact,
    package_metadata: dict
) -> Package:
    """
    Publishes an artifact through secure distribution
    
    Args:
        chain: Configured supply chain
        artifact: Built artifact to publish
        package_metadata: Package metadata
        
    Returns:
        Published package
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Test code
metadata = {
    "name": "secure-package",
    "version": "1.0.0",
    "author": "test-dev"
}

# Publish artifact
package = publish_secure_artifact(supply_chain, artifact, metadata)

# Verify package was published securely
assert package.name == metadata["name"]
assert package.version == metadata["version"]
assert package.signature is not None

# Check transparency log
log = supply_chain.distribution.transparency_log
found = False
for entry in log.entries:
    if entry["package"] == package.name:
        found = True
        break
assert found

# Verify repository
repo = supply_chain.distribution.repository
assert package.name in repo.packages
assert package.version in repo.packages[package.name]

print("Secure distribution pipeline implemented successfully!")

<details>
<summary>Solution</summary>

```python
def publish_secure_artifact(
    chain: SecureSupplyChain,
    artifact: BuildArtifact,
    package_metadata: dict
) -> Package:
    # Verify artifact signature
    if not verify_artifact_signature(
        artifact=artifact,
        trusted_keys=chain.build.system.trusted_keys
    ):
        raise ValueError("Invalid artifact signature")
        
    # Verify SLSA provenance
    if not verify_slsa_provenance(
        artifact=artifact,
        provenance=artifact.metadata["slsa_provenance"],
        policies=chain.security_policies["build"]
    ):
        raise ValueError("Invalid SLSA provenance")
        
    # Create package
    package = Package(
        name=package_metadata["name"],
        version=package_metadata["version"],
        content=artifact.content,
        author=package_metadata["author"],
        dependencies=package_metadata.get("dependencies", {})
    )
    
    # Sign package
    package.signature = sign_package(
        package=package,
        key=get_signing_key(chain)
    )
    
    # Add to transparency log
    add_transparency_log_entry(
        log=chain.distribution.transparency_log,
        package=package,
        artifact=artifact
    )
    
    # Publish to repository
    publish_to_repository(
        repo=chain.distribution.repository,
        package=package
    )
    
    # Verify mirrors are consistent
    verify_mirror_consistency(
        package=package,
        repository=chain.distribution.repository
    )
    
    return package
```
</details>

### Exercise 4: Security Validation

Finally, implement validation functions that verify the security of the pipeline:
1. Attempt various attacks
2. Verify all attestations
3. Check policy compliance
4. Validate transparency log

This ensures the security controls are working as intended.

In [ ]:
def validate_supply_chain_security(chain: SecureSupplyChain) -> List[str]:
    """
    Validates security of the supply chain
    
    Args:
        chain: Supply chain to validate
        
    Returns:
        List of validation failures, empty if all pass
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Test code
# Add test data
test_commit = SignedGitCommit(
    hash="def456",
    message="Test commit 2", 
    timestamp=datetime.now(),
    author="test-dev",
    signature=b"valid-sig-2",
    author_public_key="test-key"
)

supply_chain.source.signed_commits.append(test_commit)

test_artifact = BuildArtifact(
    content=b"test artifact",
    metadata={
        "source_commit": test_commit.hash,
        "signature": b"valid-sig-3",
        "slsa_provenance": {}
    }
)
supply_chain.build.artifacts["test"] = test_artifact

# Validate security
failures = validate_supply_chain_security(supply_chain)

# Should detect various issues
assert any("unsigned commit" in f for f in failures)
assert any("invalid provenance" in f for f in failures)
assert any("missing attestation" in f for f in failures)

# Fix issues and validate again
supply_chain.source.trusted_keys.add("test-key")
test_artifact.metadata["slsa_provenance"] = {"valid": "provenance"}
supply_chain.build.system.trusted_keys.add("build-key")

failures = validate_supply_chain_security(supply_chain)
assert len(failures) == 0

print("Security validation implemented successfully!")

<details>
<summary>Solution</summary>

```python
def validate_supply_chain_security(chain: SecureSupplyChain) -> List[str]:
    failures = []
    
    # Validate source control
    for commit in chain.source.signed_commits:
        # Check commit signatures
        if not verify_commit_signature(commit, chain.source.trusted_keys):
            failures.append(f"Unsigned commit: {commit.hash}")
            
        # Check protected branches
        if commit.branch in chain.security_policies["source"]["protected_branches"]:
            if not verify_protected_branch_rules(commit, chain.security_policies):
                failures.append(f"Protected branch violation: {commit.branch}")
                
    # Validate builds
    for artifact_id, artifact in chain.build.artifacts.items():
        # Verify build signatures
        if not verify_artifact_signature(artifact, chain.build.system.trusted_keys):
            failures.append(f"Unsigned artifact: {artifact_id}")
            
        # Verify SLSA provenance
        if not verify_slsa_provenance(artifact, artifact.metadata.get("slsa_provenance", {})):
            failures.append(f"Invalid provenance: {artifact_id}")
            
        # Verify hermetic builds
        if not verify_hermetic_build(artifact, chain.security_policies["build"]):
            failures.append(f"Non-hermetic build: {artifact_id}")
            
    # Validate distribution
    repo = chain.distribution.repository
    for pkg_name, versions in repo.packages.items():
        for version, package in versions.items():
            # Verify package signatures
            if not verify_package_signature(package, repo.trusted_publishers):
                failures.append(f"Unsigned package: {pkg_name}@{version}")
                
            # Verify transparency log
            if not verify_transparency_log_entry(package, chain.distribution.transparency_log):
                failures.append(f"Missing log entry: {pkg_name}@{version}")
                
            # Verify attestations
            if not verify_package_attestations(package, chain.security_policies["distribution"]):
                failures.append(f"Missing attestation: {pkg_name}@{version}")
                
    # Attempt bypass attacks
    failures.extend(attempt_security_bypasses(chain))
    
    return failures
```
</details>